In this homework you will use code in a Jupyter notebook to fetch articles from Wikipedia and strip out the tags using “Beautiful Soup” (see accompanying file data_fetch.py). Then you will use keras to create an RNN model which can generate an altogether new sentence based on the body of content associated articles.


##1.Run the code provided in the data_fetch.py and insert your own Wikipedia article URL to retrieve a list of documents representing Wikipedia articles linked from the one provided.


In [ ]:
import bs4 as bs
import urllib.request
import re


# Place the URL to your wikipedia page here:
##########################################################
wiki_url = 'https://en.wikipedia.org/wiki/Thomas_Jefferson'
##########################################################


def extract_wikipedia_links(url):
  """
  Returns a list of linked wikipedia urls from an initial wikipedia page
  """
  data = urllib.request.urlopen(url)
  article = data.read()
  html_article = bs.BeautifulSoup(article,'html')

  hrefs = [tag.get('href') for tag in html_article.find_all('a', {'href': re.compile('/wiki/[a-zA-Z]')})]
  hrefs_pattern1 = re.compile('.*:.*')
  hrefs_pattern2 = re.compile('.*\/\/')
  wiki_hrefs = list(set(
      ['https://en.wikipedia.org'+href for href in hrefs if not hrefs_pattern1.match(
          href) and not hrefs_pattern2.match(href)]))
  return wiki_hrefs



def extract_wikipedia_text(url):
  """
  Returns the text from the body of a wikepedia article
  """
  data = urllib.request.urlopen(url)
  article = data.read()
  html_article = bs.BeautifulSoup(article,'html')
  paragraphs = html_article.find_all('p')
  paragraph_list = []
  pattern_1 = re.compile('\[[0-9]*\]')
  pattern_2 = re.compile('\s+')
  for p in paragraphs:
    paragraph_text = p.text
    clean_paragraph_text = re.sub(
                              pattern_2, ' ', re.sub(
                                  pattern_1, '', paragraph_text))
    paragraph_list.append(clean_paragraph_text)
  if len(paragraph_list) == 0:
    return ''
  return paragraph_list[0] + ' '.join(paragraph_list[1:])



def get_wiki_content(url):
  """
  Returns a list of texts from the body of all linked wikepedia articles
  """
  wiki_links = extract_wikipedia_links(url)
  wiki_text = []
  
  for link in wiki_links:
    text = extract_wikipedia_text(link)
    wiki_text.append(text)

  return wiki_text

wiki_text_list = get_wiki_content(wiki_url)


In [ ]:
paras= extract_wikipedia_text(wiki_url)


##2.Load the pre-trained spaCy pipeline called “en_core_web_lg” and then import and load it. Note that the pipeline takes two or three minutes to download because of the large amount of word embedding data.

In [ ]:

import spacy.cli # use the command line interface
spacy.cli.download("en_core_web_lg") # this import the large model onto your virtual machine
import en_core_web_lg
nlp_lg = en_core_web_lg.load()

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')



##3.Pass each sentence into separate NLP documents via the spaCy pipeline. Do not remove any words or punctuation.


In [ ]:
import numpy as np

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
sentences = nltk.sent_tokenize(paras)
array1 = []
array2_meanVector = []
array_test=[]
len(sentences)

699

In [ ]:
for z in nlp_lg(sentences[0]):
    array1.append(nlp_lg.vocab["z"].vector)
meaVector = np.mean(array1, axis=0)


In [ ]:
str(nlp_lg(sentences[0])[1])

'Thomas'

In [ ]:
for k in range(0,len(sentences)):
  for z in range(0,len(nlp_lg(sentences[k]))):
    array1.append(nlp_lg.vocab[str(nlp_lg(sentences[k])[z])].vector)
  meanVector = np.mean(array1, axis=0)
  array1=[]
  array2_meanVector.append(meanVector)

In [ ]:
np.mean(meanVector)

0.005496397


##4.Compute the mean word vector across all the word vectors in each document (refer to Homework 3).


In [ ]:
# array2_meanVector is the mean vector across all the documents, calculated above
len(array2_meanVector)

699


##5.Use cosine similarity to create similarity scores between each pair of mean word vectors. You should end up with floating point values bounded between 0 and 1 for every pair of vectors. Make sure to avoid comparing a word vector to itself.


In [ ]:
from scipy.spatial.distance import cosine
def cosine(variableA,variableB):
    return np.dot(variableA,variableB) / (np.sqrt(np.dot(variableA,variableA)) * np.sqrt(np.dot(variableB,variableB)))

In [ ]:
len(array2_meanVector)

699


##6.Interpret the results from the previous step. Determine the two Wikipedia articles with the lowest cosine similarity and the two articles with the greatest cosine similarity scores. Which two articles are the most similar to each other? Which two are least similar? Does the pattern of values make sense? Why or why not?

In [ ]:
import pandas as pd
cosine_values_arry = []
freeza=[[]]

In [ ]:
for k in range(0,len(array2_meanVector)):
  for z in range(0,len(array2_meanVector)):
    if(k!=z):
      cosine_values_arry.append(cosine(array2_meanVector[k],array2_meanVector[z]))
  freeza.append(cosine_values_arry)
  #print(cosine_values_arry)
  cosine_values_arry = []

In [ ]:
df= pd.DataFrame(freeza)
min_arry =[]
max_arry =[]

In [ ]:
for i in range(0,692):
  min_arry.append(df[i].min(skipna=True))
  max_arry.append(df[i].max(skipna=True))

In [ ]:
min(min_arry)

0.2584898769855499

In [ ]:
max(max_arry)

0.994910717010498


##7.Create ragged tensors of individual characters as well as of index IDs (see lab 9) for all .


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers.experimental import preprocessing
tf.executing_eagerly()

True

In [ ]:
# Confirms that preprocessing.StringLookup is an ABC and not an instance
type(preprocessing.StringLookup), isinstance(preprocessing.StringLookup, preprocessing.StringLookup)

(abc.ABCMeta, False)

We calculated paras at the start of this notebook

In [ ]:
parase_set = set(paras)
vocab = sorted(set(paras))

In [ ]:
chars = tf.strings.unicode_split(list(vocab), input_encoding='UTF-8')

In [ ]:
chars

<tf.RaggedTensor [[b' '], [b'!'], [b'"'], [b'$'], [b'&'], [b"'"], [b'('], [b')'], [b','], [b'-'], [b'.'], [b'/'], [b'0'], [b'1'], [b'2'], [b'3'], [b'4'], [b'5'], [b'6'], [b'7'], [b'8'], [b'9'], [b':'], [b';'], [b'?'], [b'A'], [b'B'], [b'C'], [b'D'], [b'E'], [b'F'], [b'G'], [b'H'], [b'I'], [b'J'], [b'K'], [b'L'], [b'M'], [b'N'], [b'O'], [b'P'], [b'Q'], [b'R'], [b'S'], [b'T'], [b'U'], [b'V'], [b'W'], [b'X'], [b'Y'], [b'Z'], [b'['], [b']'], [b'a'], [b'b'], [b'c'], [b'd'], [b'e'], [b'f'], [b'g'], [b'h'], [b'i'], [b'j'], [b'k'], [b'l'], [b'm'], [b'n'], [b'o'], [b'p'], [b'q'], [b'r'], [b's'], [b't'], [b'u'], [b'v'], [b'w'], [b'x'], [b'y'], [b'z'], [b'\xc3\xa7'], [b'\xc3\xa8'], [b'\xc3\xa9'], [b'\xc3\xaa'], [b'\xc3\xb4'], [b'\xe2\x80\x93'], [b'\xe2\x80\x94']]>

In [ ]:
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [64], [65], [66], [67], [68], [69], [70], [71], [72], [73], [74], [75], [76], [77], [78], [79], [80], [81], [82], [83], [84], [85], [86]]>


##8.Build a Recurrent Neural Network (RNN) model using keras with the following parameters:
##-vocab_size = len(vocab)
##-embedding_dim = 256 #
##-rnn_units = 1024


In [ ]:
# creating training examples and targets
all_ids = ids_from_chars(tf.strings.unicode_split(paras,'UTF-8'))
type(all_ids)

tensorflow.python.framework.ops.EagerTensor

In [ ]:
all_ids.get_shape()
# This creates a dataset whose elements are slices from the original tensor
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
type(ids_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [ ]:
# commenting it to avoid the long output
#temp = ids_dataset.unique()
#for element in temp:
#  print(element.numpy())

In [ ]:
len(vocab)

86

In [ ]:
vocab_size = len(vocab)
embedding_dim =256
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)

    # What's this layer?
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    
    # What's this layer?
    self.rnn = tf.keras.layers.SimpleRNN(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    
    # What's this layer?
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.rnn.get_initial_state(x)
    x, states = self.rnn(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
type(model)

__main__.MyModel


##9.Train your RNN for 20 EPOCHS.


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
seq_length = 80 # About one line of standard text

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset:
# TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements.
EPOCHS = 20
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))


In [ ]:
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/20
18/18 [==============================] - 23s 1s/step - loss: 3.5391
Epoch 2/20
18/18 [==============================] - 21s 1s/step - loss: 3.1140
Epoch 3/20
18/18 [==============================] - 21s 1s/step - loss: 2.9246
Epoch 4/20
18/18 [==============================] - 21s 1s/step - loss: 2.6933
Epoch 5/20
18/18 [==============================] - 21s 1s/step - loss: 2.5501
Epoch 6/20
18/18 [==============================] - 21s 1s/step - loss: 2.4342
Epoch 7/20
18/18 [==============================] - 21s 1s/step - loss: 2.3568
Epoch 8/20
18/18 [==============================] - 21s 1s/step - loss: 2.2897
Epoch 9/20
18/18 [==============================] - 21s 1s/step - loss: 2.2277
Epoch 10/20
18/18 [==============================] - 21s 1s/step - loss: 2.1755
Epoch 11/20
18/18 [==============================] - 21s 1s/step - loss: 2.1245
Epoch 12/20
18/18 [==============================] - 21s 1s/step - loss: 2.0745
Epoch 13/20
18/18 [==============================


##10. Generate a new sentence based on the original Wikipedia article you chose and seed the model with the first three words in the first sentence of the article.

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
import time

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Thomas_Jefferson was'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Thomas_Jefferson was ace mase plences, arch duilary. On the S.S. Jatus an 1801. Oe duthy ardemste castral towend Univelsatevingluw tam Store. Encompariagly, and there Sawer conturlan. The resurnctory and proves Marmyen Castilite in manaye tox atonn recorn and and erpontand tot Burtis gromolit ond abaic as to 1706, Jefferson a the State of Statealis. Montices, relowes cerianirg binlica, "piyta Pancied the Freendent of his now tre teed fabed ". 170k wall and he toll fiden beri, in 1906, Jefferson was cralia espaise-viling, "th-sepaloniry effreds in tied the Renellion for in Nather by whing utley He itsy and Franct merkning on Ix 1774, Burs cellanc into-ualoro-somitiag ith ol the alinay to rigltanth o pralis, a s arcessming Corminicy on 1104, Share to pround, and of camte soputain sationed Wiitas. Slarch (derawas and parsho and ftribling but pliburqy in the secay and vilieve be his astorized bottry enminding wimh EnS6 polis lew Mantuan, Jefferson bat noth Matist appisciften tho Decial in 

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Thomas_Jefferson was', ' Founding Father who', 'As president, Jefferson'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'Thomas_Jefferson was. Averivach inT180 Eed cassitices, ente more\xe2\x80\x93"tion lisiated Tongress bnizess, ther an thror. .0. fele and the iale is destrouncembear\'s in 1803 and orsa pupligation was a labe Br the .0. Aht of his enfiter, he aftecredonitall Manrot appostof copisillutitizes proent brock mideened thay, After Adexulleatith grection\'s Congrce by ade doan of the Lacy of lencead, th cener Paned, the Twote his dould Or vilatory Comme, Helinial inder Marizanized at id Lacued repyed the Mings of the UX2. The agco, his menber nears, in dill bsong leass and was a preainal Conmmargm. His liftite of Vircanian, at Jefferson. In 1648 eeslasad Colitien.  Ithe "AcCpadine Prssinial ammate" Selitsoul in Jeltoul S.S presitert songery fouts lat ase Ecsoush mad y arg of tie cour Peseraced as hingltures as dait nevern tiatlly. Wil the inent an  on N0-Ne Virginia one to rolomien act was elte ens a utialutict, EnCinetteas hawern[stronand thaigt of 1754) his oble namonty, an aint